In [1]:
from pathlib import Path
import tomllib
from oauthlib.oauth2 import BackendApplicationClient
from requests_oauthlib import OAuth2Session
import blizzapi

In [2]:
client_filepath = Path(r"D:\Programming\github-repos\nop-raid-bot\clientdetails.toml")
with open(client_filepath, "rb") as client_file:
    client_data = tomllib.load(client_file)

CLIENT_ID = client_data["client"]["id"]
CLIENT_SECRET = client_data["client"]["secret"]

TOKEN_URL = "https://oauth.battle.net/token"

In [3]:
client = BackendApplicationClient(client_id=CLIENT_ID)
oauth = OAuth2Session(client=client, state="blah123")
token = oauth.fetch_token(token_url=TOKEN_URL, client_id=CLIENT_ID, client_secret=CLIENT_SECRET)

In [4]:
testchar = blizzapi.Character("Aptosaurinae", "Draenor")
blizz_urls = blizzapi.BlizzardAPIURLs()
chardata = blizzapi.CharacterData(testchar, blizz_api_urls=blizz_urls, oauth=oauth)
chardata.get_specific_raid_data("The War Within", "Nerub-ar Palace", "Heroic")

"Aptosaurinae-Draenor:\n- Heroic_Ulgrax the Devourer: <t:1730661349:D>\n- Heroic_The Bloodbound Horror: <t:1730662050:D>\n- Heroic_Sikran, Captain of the Sureki: <t:1730662840:D>\n- Heroic_Rasha'nan: <t:1730663569:D>\n- Heroic_Broodtwister Ovi'nax: <t:1730664631:D>\n- Heroic_Nexus-Princess Ky'veza: <t:1730667103:D>"

In [5]:
chardata.get_specific_raid_data("The War Within", "Nerub-ar Palace", "Mythic")

d:\Programming\github-repos\nop-raid-bot\blizzapi.py:238: UserWarning: No data found for Nerub-ar Palace [Mythic]
  warnings.warn(f"No data found for {raid_name} [{difficulty}]")


'Aptosaurinae-Draenor:\nNone'

In [6]:
chardata._get_specific_raid_data("The War Within", "Nerub-ar Palace", "Heroic")

CharacterName,RealmName,Heroic_Ulgrax the Devourer,Heroic_The Bloodbound Horror,"Heroic_Sikran, Captain of the Sureki",Heroic_Rasha'nan,Heroic_Broodtwister Ovi'nax,Heroic_Nexus-Princess Ky'veza
str,str,i32,i32,i32,i32,i32,i32
"""Aptosaurinae""","""Draenor""",1730661349,1730662050,1730662840,1730663569,1730664631,1730667103


In [7]:
chardata.get_current_enchants(True)

"Enchants:\n- 7 / 8\n- Off Hand missing enchant\n- Back: Enchanted: +355 Avoidance :quality3:\n- Chest: Enchanted: Crystalline Radiance :quality2:\n- Wrist: Enchanted: +710 Avoidance :quality3:\n- Legs: Enchanted: +710 Avoidance :quality3:\n- Feet: Enchanted: Scout's March :quality3:\n- Ring 1: Enchanted: +190 Haste :quality3:\n- Ring 2: Enchanted: +190 Haste :quality3:\n- Main Hand: Enchanted: Stormrider's Fury :quality3:"

In [8]:
equipment = oauth.get(blizz_urls.get_equipment(testchar)).json()
equipment

{'_links': {'self': {'href': 'https://eu.api.blizzard.com/profile/wow/character/draenor/aptosaurinae/equipment?namespace=profile-eu'}},
 'character': {'key': {'href': 'https://eu.api.blizzard.com/profile/wow/character/draenor/aptosaurinae?namespace=profile-eu'},
  'name': 'Aptosaurinae',
  'id': 177892318,
  'realm': {'key': {'href': 'https://eu.api.blizzard.com/data/wow/realm/506?namespace=dynamic-eu'},
   'name': 'Draenor',
   'id': 506,
   'slug': 'draenor'}},
 'equipped_items': [{'item': {'key': {'href': 'https://eu.api.blizzard.com/data/wow/item/212092?namespace=static-11.0.5_56865-eu'},
    'id': 212092},
   'sockets': [{'socket_type': {'type': 'PRISMATIC',
      'name': 'Prismatic Socket'},
     'item': {'key': {'href': 'https://eu.api.blizzard.com/data/wow/item/213746?namespace=static-11.0.5_56865-eu'},
      'name': 'Elusive Blasphemite',
      'id': 213746},
     'display_string': '+181 Primary Stat and +2% Movement Speed per unique Algari gem color',
     'media': {'key': {'

In [9]:
[item["slot"]["name"] for item in equipment["equipped_items"]]

['Head',
 'Neck',
 'Shoulders',
 'Shirt',
 'Chest',
 'Waist',
 'Legs',
 'Feet',
 'Wrist',
 'Hands',
 'Ring 1',
 'Ring 2',
 'Trinket 1',
 'Trinket 2',
 'Back',
 'Main Hand',
 'Tabard']

In [10]:
encounters_url = blizz_urls.get_encounter_journal_index()
nerubar_url = blizz_urls.get_encounter_list(1273)
encounters_json = oauth.get(encounters_url).json()
nerubar_json = oauth.get(nerubar_url).json()
display(encounters_json, nerubar_json)

{'_links': {'self': {'href': 'https://eu.api.blizzard.com/data/wow/journal-instance/?namespace=static-11.0.5_56865-eu'}},
 'instances': [{'key': {'href': 'https://eu.api.blizzard.com/data/wow/journal-instance/64?namespace=static-11.0.5_56865-eu'},
   'name': 'Shadowfang Keep',
   'id': 64},
  {'key': {'href': 'https://eu.api.blizzard.com/data/wow/journal-instance/67?namespace=static-11.0.5_56865-eu'},
   'name': 'The Stonecore',
   'id': 67},
  {'key': {'href': 'https://eu.api.blizzard.com/data/wow/journal-instance/76?namespace=static-11.0.5_56865-eu'},
   'name': "Zul'Gurub",
   'id': 76},
  {'key': {'href': 'https://eu.api.blizzard.com/data/wow/journal-instance/66?namespace=static-11.0.5_56865-eu'},
   'name': 'Blackrock Caverns',
   'id': 66},
  {'key': {'href': 'https://eu.api.blizzard.com/data/wow/journal-instance/63?namespace=static-11.0.5_56865-eu'},
   'name': 'Deadmines',
   'id': 63},
  {'key': {'href': 'https://eu.api.blizzard.com/data/wow/journal-instance/69?namespace=stati

{'_links': {'self': {'href': 'https://eu.api.blizzard.com/data/wow/journal-instance/1273?namespace=static-11.0.5_56865-eu'}},
 'id': 1273,
 'name': 'Nerub-ar Palace',
 'map': {'name': 'Nerub-ar Palace', 'id': 2657},
 'description': "Deep within the kingdom of Azj-Kahet lies Nerub-ar Palace, Queen Ansurek's seat of power. From this shadowed throne Queen Ansurek has tightened her grip on her Sureki followers, leading them along a dark path to a new future. As her paranoia grows, the members of the conspiratorial Severed Threads launch their final gambit against Ansurek's rule.",
 'encounters': [{'key': {'href': 'https://eu.api.blizzard.com/data/wow/journal-encounter/2607?namespace=static-11.0.5_56865-eu'},
   'name': 'Ulgrax the Devourer',
   'id': 2607},
  {'key': {'href': 'https://eu.api.blizzard.com/data/wow/journal-encounter/2611?namespace=static-11.0.5_56865-eu'},
   'name': 'The Bloodbound Horror',
   'id': 2611},
  {'key': {'href': 'https://eu.api.blizzard.com/data/wow/journal-enc

In [11]:
raid_query = blizz_urls.get_raids(testchar)
raid_query

'https://eu.api.blizzard.com/profile/wow/character/draenor/aptosaurinae/encounters/raids?locale=en_GB&namespace=profile-eu'

In [12]:
response = oauth.get(raid_query)
response

<Response [200]>

In [13]:
response.headers

{'Date': 'Mon, 04 Nov 2024 22:47:06 GMT', 'Content-Type': 'application/json;charset=UTF-8', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Battlenet-Schema-Revision': '2', 'Last-Modified': 'Mon, 4 Nov 2024 21:18:00 GMT', 'Battlenet-Namespace': 'profile-eu', 'vary': 'accept-encoding', 'blizzard-token-expires': '2024-11-05T22:47:04.366Z', 'Battlenet-Schema': 'character-raid-progression-raids', 'content-encoding': 'gzip', 'x-trace-traceid': '00a630e3-c3c2-3474-8f34-577402de02a5', 'x-trace-spanid': '1801b920-a197-0434-ee3f-fa165b445d20', 'x-trace-parentspanid': '1801b920-a197-0404-ee3f-fa165b445d20', 'x-frame-options': 'SAMEORIGIN', 'X-Content-Type-Options': 'nosniff', 'server': 'blizzard'}

In [14]:
response.json()

{'_links': {'self': {'href': 'https://eu.api.blizzard.com/profile/wow/character/draenor/aptosaurinae/encounters/raids?namespace=profile-eu'}},
 'character': {'key': {'href': 'https://eu.api.blizzard.com/profile/wow/character/draenor/aptosaurinae?namespace=profile-eu'},
  'name': 'Aptosaurinae',
  'id': 177892318,
  'realm': {'key': {'href': 'https://eu.api.blizzard.com/data/wow/realm/506?namespace=dynamic-eu'},
   'name': 'Draenor',
   'id': 506,
   'slug': 'draenor'}},
 'expansions': [{'expansion': {'key': {'href': 'https://eu.api.blizzard.com/data/wow/journal-expansion/73?namespace=static-11.0.5_56865-eu'},
    'name': 'Cataclysm',
    'id': 73},
   'instances': [{'instance': {'key': {'href': 'https://eu.api.blizzard.com/data/wow/journal-instance/72?namespace=static-11.0.5_56865-eu'},
      'name': 'The Bastion of Twilight',
      'id': 72},
     'modes': [{'difficulty': {'type': 'LEGACY_25_MAN_HEROIC',
        'name': '25 Player (Heroic)'},
       'status': {'type': 'COMPLETE', 'nam

In [15]:
response.json()["expansions"]

[{'expansion': {'key': {'href': 'https://eu.api.blizzard.com/data/wow/journal-expansion/73?namespace=static-11.0.5_56865-eu'},
   'name': 'Cataclysm',
   'id': 73},
  'instances': [{'instance': {'key': {'href': 'https://eu.api.blizzard.com/data/wow/journal-instance/72?namespace=static-11.0.5_56865-eu'},
     'name': 'The Bastion of Twilight',
     'id': 72},
    'modes': [{'difficulty': {'type': 'LEGACY_25_MAN_HEROIC',
       'name': '25 Player (Heroic)'},
      'status': {'type': 'COMPLETE', 'name': 'Complete'},
      'progress': {'completed_count': 5,
       'total_count': 5,
       'encounters': [{'encounter': {'key': {'href': 'https://eu.api.blizzard.com/data/wow/journal-encounter/156?namespace=static-11.0.5_56865-eu'},
          'name': 'Halfus Wyrmbreaker',
          'id': 156},
         'completed_count': 1,
         'last_kill_timestamp': 1696671210000},
        {'encounter': {'key': {'href': 'https://eu.api.blizzard.com/data/wow/journal-encounter/157?namespace=static-11.0.5_56

In [16]:
[data["expansion"]["name"] for data in response.json()["expansions"]]

['Cataclysm',
 'Warlords of Draenor',
 'Dragonflight',
 'The War Within',
 'Current Season']

In [17]:
expansions_list = [data["expansion"]["name"] for data in response.json()["expansions"]]
exp_idx = expansions_list.index("The War Within")
instance_names = [data["instance"]["name"] for data in response.json()["expansions"][exp_idx]["instances"]]
ins_idx = instance_names.index("Nerub-ar Palace")
difficulities = [data["difficulty"]["name"] for data in response.json()["expansions"][exp_idx]["instances"][ins_idx]["modes"]]
difficulty_index = difficulities.index("Heroic")

In [18]:
response.json()["expansions"][exp_idx]["instances"][ins_idx]["modes"][difficulty_index]["progress"]

{'completed_count': 6,
 'total_count': 8,
 'encounters': [{'encounter': {'key': {'href': 'https://eu.api.blizzard.com/data/wow/journal-encounter/2607?namespace=static-11.0.5_56865-eu'},
    'name': 'Ulgrax the Devourer',
    'id': 2607},
   'completed_count': 2,
   'last_kill_timestamp': 1730661349000},
  {'encounter': {'key': {'href': 'https://eu.api.blizzard.com/data/wow/journal-encounter/2611?namespace=static-11.0.5_56865-eu'},
    'name': 'The Bloodbound Horror',
    'id': 2611},
   'completed_count': 2,
   'last_kill_timestamp': 1730662050000},
  {'encounter': {'key': {'href': 'https://eu.api.blizzard.com/data/wow/journal-encounter/2599?namespace=static-11.0.5_56865-eu'},
    'name': 'Sikran, Captain of the Sureki',
    'id': 2599},
   'completed_count': 3,
   'last_kill_timestamp': 1730662840000},
  {'encounter': {'key': {'href': 'https://eu.api.blizzard.com/data/wow/journal-encounter/2609?namespace=static-11.0.5_56865-eu'},
    'name': "Rasha'nan",
    'id': 2609},
   'completed